In [1]:
import pandas as pd
import requests
import telebot
from datetime import date, datetime, timedelta
import datetime as dt
from workadays import workdays as wd
from time import sleep
import holidays
import MetaTrader5 as mt5


In [50]:
mt5.initialize()

True

In [54]:
ativo = 'VALE3'
vencimento = '2024-03-15'
ticker_selecionado = mt5.symbol_select(ativo)
ticker_info = mt5.symbol_info_tick(ativo)
preco_ativo = ticker_info.last
preco_ativo

66.88

In [53]:
try:
    mt5.symbol_select(ativo,True)
    ticker_info = mt5.symbol_info_tick(ativo)
    preco_ativo = ticker_info.last
    #preco_ativo = round(yf.download(ativo +'.SA')['Adj Close'].iloc[-1], 2)
except Exception as e:
    print(f"Erro ao baixar dados para {ativo}: {e}.")
    pass
preco_ativo

66.88

In [55]:
try:
    url = f'https://opcoes.net.br/listaopcoes/completa?idAcao={ativo}&listarVencimentos=false&cotacoes=true&vencimentos={vencimento}'
    r = requests.get(url).json()
    x = ([ativo, vencimento, i[0].split('_')[0], i[2], i[5], i[9], i[10]] for i in r['data']['cotacoesOpcoes'])
    df = pd.DataFrame(x, columns=['Ativo', 'Vencimento', 'Ticker', 'Tipo', 'Strike', 'Negócios', 'Volume'])
except Exception as e:
    print(f"Erro ao baixar dados da opcoes.net.br: {e}.")
    pass
df

,Ativo,Vencimento,Ticker,Tipo,Strike,Negócios,Volume
0,VALE3,2024-03-15,VALEC1,CALL,104.32,4.0,1000.0
1,VALE3,2024-03-15,VALEC10,CALL,102.32,2.0,3.0
2,VALE3,2024-03-15,VALEC84,CALL,80.32,1.0,2.0
3,VALE3,2024-03-15,VALEC85,CALL,82.82,2.0,50.0
4,VALE3,2024-03-15,VALEC100,CALL,100.32,2.0,513.0
...,...,...,...,...,...,...,...
185,VALE3,2024-03-15,VALEO953,PUT,95.32,8.0,584597.0
186,VALE3,2024-03-15,VALEO960,PUT,93.82,1.0,52250.0
187,VALE3,2024-03-15,VALEO963,PUT,96.32,1.0,8850.0
188,VALE3,2024-03-15,VALEO973,PUT,97.32,1.0,93000.0


In [62]:
df_put = df[df['Tipo'] == 'PUT']
df_put_op = df_put.copy()
for index, row in df_put_op.iterrows():
    ticker = row['Ticker']
    # Verifica se o ticker é válido e se o preço de ask está disponível
    ticker_selecionado = mt5.symbol_select(ticker,True)
    ticker_info = mt5.symbol_info_tick(ticker)
    
    if ticker_info and ticker_info.time:
        # Arredonda o preço de ask para o número inteiro mais próximo
        df_put_op.at[index, 'Preço'] = ticker_info.ask
        df_put_op.at[index, 'Último Negócio'] = datetime.fromtimestamp(ticker_info.time).date()
        #df.at[index, 'Volume Dia'] = ticker_info.volume
    else:
        df_put_op.at[index, 'Preço'] = 0.0
df_put_op

,Ativo,Vencimento,Ticker,Tipo,Strike,Negócios,Volume,Preço,Último Negócio
95,VALE3,2024-03-15,VALEO1,PUT,104.32,3.0,37344.0,0.0,2024-03-01
96,VALE3,2024-03-15,VALEO10,PUT,102.32,3.0,238770.0,0.0,2024-02-28
97,VALE3,2024-03-15,VALEO84,PUT,80.32,2.0,24823.0,0.0,2024-02-29
98,VALE3,2024-03-15,VALEO85,PUT,82.82,1.0,68000.0,0.0,2024-02-28
99,VALE3,2024-03-15,VALEO100,PUT,100.32,NaN,NaN,0.0,2024-02-28
...,...,...,...,...,...,...,...,...,...
185,VALE3,2024-03-15,VALEO953,PUT,95.32,8.0,584597.0,0.0,2024-03-01
186,VALE3,2024-03-15,VALEO960,PUT,93.82,1.0,52250.0,0.0,2024-02-28
187,VALE3,2024-03-15,VALEO963,PUT,96.32,1.0,8850.0,0.0,2024-03-01
188,VALE3,2024-03-15,VALEO973,PUT,97.32,1.0,93000.0,0.0,2024-02-28


In [63]:
df_put_op.rename(columns={
        'Ticker': 'Ticker Put',
        'Tipo': 'Tipo Put',
        'Strike': 'Strike Put', 
        'Preço': 'Preço Put',
        'Negócios': 'Negócios Put',
        'Volume': 'Volume Put',
        'Último Negócio': 'Último Negócio Put'
    }, inplace=True)
df_put_op

,Ativo,Vencimento,Ticker Put,Tipo Put,Strike Put,Negócios Put,Volume Put,Preço Put,Último Negócio Put
95,VALE3,2024-03-15,VALEO1,PUT,104.32,3.0,37344.0,0.0,2024-03-01
96,VALE3,2024-03-15,VALEO10,PUT,102.32,3.0,238770.0,0.0,2024-02-28
97,VALE3,2024-03-15,VALEO84,PUT,80.32,2.0,24823.0,0.0,2024-02-29
98,VALE3,2024-03-15,VALEO85,PUT,82.82,1.0,68000.0,0.0,2024-02-28
99,VALE3,2024-03-15,VALEO100,PUT,100.32,NaN,NaN,0.0,2024-02-28
...,...,...,...,...,...,...,...,...,...
185,VALE3,2024-03-15,VALEO953,PUT,95.32,8.0,584597.0,0.0,2024-03-01
186,VALE3,2024-03-15,VALEO960,PUT,93.82,1.0,52250.0,0.0,2024-02-28
187,VALE3,2024-03-15,VALEO963,PUT,96.32,1.0,8850.0,0.0,2024-03-01
188,VALE3,2024-03-15,VALEO973,PUT,97.32,1.0,93000.0,0.0,2024-02-28


In [57]:

    #df_put_op['Último Negócio'] = pd.to_datetime(df_put_op['Último Negócio'], format="%d/%m/%Y")
    df_put_op.rename(columns={
        'Ticker': 'Ticker Put',
        'Tipo': 'Tipo Put',
        'Strike': 'Strike Put', 
        'Preço': 'Preço Put',
        'Negócios': 'Negócios Put',
        'Volume': 'Volume Put',
        'Último Negócio': 'Último Negócio Put'
    }, inplace=True)
except Exception as e:
    print(f"Erro ao criar a lista de PUTs: {e}.")
    pass
df_put_op

,Ativo,Vencimento,Ticker Put,Tipo Put,Strike Put,Negócios Put,Volume Put,Preço Put,Último Negócio Put
95,VALE3,2024-03-15,VALEO1,PUT,104.32,3.0,37344.0,0.0,NaN
96,VALE3,2024-03-15,VALEO10,PUT,102.32,3.0,238770.0,0.0,NaN
97,VALE3,2024-03-15,VALEO84,PUT,80.32,2.0,24823.0,0.0,NaN
98,VALE3,2024-03-15,VALEO85,PUT,82.82,1.0,68000.0,0.0,NaN
99,VALE3,2024-03-15,VALEO100,PUT,100.32,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...
185,VALE3,2024-03-15,VALEO953,PUT,95.32,8.0,584597.0,0.0,NaN
186,VALE3,2024-03-15,VALEO960,PUT,93.82,1.0,52250.0,0.0,NaN
187,VALE3,2024-03-15,VALEO963,PUT,96.32,1.0,8850.0,0.0,NaN
188,VALE3,2024-03-15,VALEO973,PUT,97.32,1.0,93000.0,0.0,NaN


In [3]:
calcula_cdi('2024-04-19')

(1.36, 32)

In [2]:
# Coleta selic anual direto pelo BC
def calcula_cdi(vencimento):
    url_selic = f'http://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json'
    selic = pd.read_json(url_selic)
    selic['data'] = pd.to_datetime(selic['data'], dayfirst=True)
    selic.set_index('data', inplace=True)
    selic = float(selic.iloc[-1].values.item())

    # Calcula a quantidade de dias úteis e o rendimento do CDI até o vencimento
    data_hoje = dt.date.today()
    vencimento = datetime.strptime(vencimento, "%Y-%m-%d")
    dias_uteis = wd.networkdays(data_hoje, vencimento, country='BR')
    cdi_operacao = round(((1 + selic / 100) ** (dias_uteis / 252) - 1) * 100, 2)
    return cdi_operacao, dias_uteis


def coleta_opcoes(ativo, vencimento):
    # Coleta o preço do ativo com base no último fechamento
    try:
        mt5.symbol_select(ativo,True)
        ticker_info = mt5.symbol_info_tick(ativo)
        preco_ativo = ticker_info.last
        #preco_ativo = round(yf.download(ativo +'.SA')['Adj Close'].iloc[-1], 2)
    except Exception as e:
        print(f"Erro ao baixar dados para {ativo}: {e}.")
        pass
    

    # Coleta informações das opções
    try:
        url = f'https://opcoes.net.br/listaopcoes/completa?idAcao={ativo}&listarVencimentos=false&cotacoes=true&vencimentos={vencimento}'
        r = requests.get(url).json()
        x = ([ativo, vencimento, i[0].split('_')[0], i[2], i[5], i[9], i[10]] for i in r['data']['cotacoesOpcoes'])
        df = pd.DataFrame(x, columns=['Ativo', 'Vencimento', 'Ticker', 'Tipo', 'Strike', 'Negócios', 'Volume'])
    except Exception as e:
        print(f"Erro ao baixar dados da opcoes.net.br: {e}.")
        pass
    # Cria um dataframe somente com as PUTs

    try:
        df_put = df[df['Tipo'] == 'PUT']
        df_put_op = df_put.copy()
        for index, row in df_put_op.iterrows():
            ticker = row['Ticker']
            # Verifica se o ticker é válido e se o preço de ask está disponível
            ticker_selecionado = mt5.symbol_select(ticker,True)
            ticker_info = mt5.symbol_info_tick(ticker)
            
            if ticker_info and ticker_info.time:
                # Arredonda o preço de ask para o número inteiro mais próximo
                df_put_op.at[index, 'Preço'] = ticker_info.ask
                df_put_op.at[index, 'Último Negócio'] = datetime.fromtimestamp(ticker_info.time).date()
                #df.at[index, 'Volume Dia'] = ticker_info.volume
            else:
                df_put_op.at[index, 'Preço'] = 0.0
        #df_put_op['Último Negócio'] = pd.to_datetime(df_put_op['Último Negócio'], format="%d/%m/%Y")
        df_put_op.rename(columns={
            'Ticker': 'Ticker Put',
            'Tipo': 'Tipo Put',
            'Strike': 'Strike Put', 
            'Preço': 'Preço Put',
            'Negócios': 'Negócios Put',
            'Volume': 'Volume Put',
            'Último Negócio': 'Último Negócio Put'
        }, inplace=True)
    except Exception as e:
        print(f"Erro ao criar a lista de PUTs: {e}.")
        pass
    # Cria um dataframe somente com as CALLs e renomeia colunas para diferenciar,
    try:
        df_call = df[df['Tipo'] == 'CALL']
        df_call_op = df_call.copy()
        df_call_op.drop(columns=['Vencimento'], inplace=True)
        for index, row in df_call_op.iterrows():
            ticker = row['Ticker']
            # Verifica se o ticker é válido e se o preço de ask está disponível
            ticker_selecionado = mt5.symbol_select(ticker,True)
            ticker_info = mt5.symbol_info_tick(ticker)
            
            if ticker_info and ticker_info.time:
                # Arredonda o preço de ask para o número inteiro mais próximo
                df_call_op.at[index, 'Preço'] = ticker_info.bid
                df_call_op.at[index, 'Último Negócio'] = datetime.fromtimestamp(ticker_info.time).date()
                #df.at[index, 'Volume Dia'] = ticker_info.volume
            else:
                df_call_op.at[index, 'Preço'] = 0.0
        #df_call_op['Último Negócio'] = pd.to_datetime(df_call_op['Último Negócio'], format="%d/%m/%Y")
        df_call_op.rename(columns={
            'Ticker': 'Ticker Call',
            'Tipo': 'Tipo Call',
            'Strike': 'Strike Call', 
            'Preço': 'Preço Call',
            'Negócios': 'Negócios Call',
            'Volume': 'Volume Call',
            'Último Negócio': 'Último Negócio Call'
        }, inplace=True)
    except Exception as e:
        print(f"Erro ao criar a lista de CALLs: {e}.")
        pass

    # Cria um dataframe com as operações possíveis para cada PUT
    try:
        df = pd.merge(df_put_op, df_call_op, on='Ativo', suffixes=('', ' Call'))
        df['Preço Ativo'] = preco_ativo
    except Exception as e:
        print(f"Erro ao criar a lista com todas as operações: {e}.")
        pass
    
    return df, df_put, df_call, preco_ativo

# Operação - Collar
def collar(ativo, vencimento, operacao='collar_alta', quantidade = 1, volume_put = 0.01, negocios_put = 1, volume_call = 0.01, 
                negocios_call = 1, filtro_data=None, risco = 0.00, corretagem_variavel = 0.00, corretagem_ordem = 0.00):
    '''
    Ativo: Ativo base da B3\n
    Vencimento: Data de vencimento da opção\n
    Operacao: collar_alta, collar_baixa\n

    '''
    # Coleta as opções disponíveis para o ativo com base no vencimento determinado
    df, df_put, df_call, preco_ativo = coleta_opcoes(ativo, vencimento)
    
    if corretagem_variavel > 0:
        tx_b3 = 0.134
    else:
        tx_b3 = 0.00
    
    if filtro_data is not None:
        filtro_data = datetime.strptime(filtro_data, '%Y-%m-%d').date()

    df['Custo Operação'] = round((preco_ativo + df['Preço Put'] - df['Preço Call']) * quantidade, 2)
    df['Corretagem'] = round((df['Custo Operação'] * ((corretagem_variavel + tx_b3) / 100)) + 6 * corretagem_ordem, 2)
    df['Corretagem (%)'] = round(df['Corretagem'] / df['Custo Operação'] * 100, 2)
    df['Risco (%)'] = float(risco)
    
    if operacao == 'collar_alta':
        df['Lucro Mínimo'] = round(df['Strike Put'] * quantidade - df['Custo Operação'], 2)
        df['Lucro Máximo'] = round(df['Strike Call'] * quantidade - df['Custo Operação'], 2)
    elif operacao == 'collar_baixa':
        df['Lucro Mínimo'] = round(df['Strike Call'] * quantidade - df['Custo Operação'], 2)
        df['Lucro Máximo'] = round(df['Strike Put'] * quantidade - df['Custo Operação'], 2)

    df['Lucro Mín (%)'] = round(df['Lucro Mínimo'] / df['Custo Operação'] * 100, 2)
    df['Lucro Máx (%)'] = round(df['Lucro Máximo'] / df['Custo Operação'] * 100, 2)
    
    # Filtra o dataframe somente com as operações lucrativas
    df_op = df.copy()
    df_op = df_op[df_op['Lucro Mín (%)'] >= df['Corretagem (%)'] + df['Risco (%)']]
    df_op = df_op[df_op['Lucro Máximo'] > 0]
    
    if operacao == 'collar_alta':
        df_op = df_op[df_op['Strike Put'] > preco_ativo]
        df_op = df_op[df_op['Strike Call'] > df_op['Strike Put']]
    elif operacao == 'collar_baixa':
        df_op = df_op[df_op['Strike Put'] < preco_ativo + 2]
        df_op = df_op[df_op['Strike Call'] < df_op['Strike Put']]

    df_op = df_op[df_op['Volume Put'] >= volume_put]
    df_op = df_op[df_op['Negócios Put'] >= negocios_put]
    df_op = df_op[df_op['Volume Call'] >= volume_call]
    df_op = df_op[df_op['Negócios Call'] >= negocios_call]
    df_op = df_op[df_op['Último Negócio Put'] >= filtro_data]
    df_op = df_op[df_op['Último Negócio Call'] >= filtro_data]
    df_op = df_op.sort_values(by='Lucro Mín (%)', ascending=False)
    df_op = df_op.drop(columns=['Tipo Put', 'Volume Put', 
                                'Tipo Call','Volume Call', 
                                'Corretagem', 'Corretagem (%)'])
    
    data_columns = ['Vencimento', 'Último Negócio Put', 'Último Negócio Call']
    for col in data_columns:
        df_op[col] = pd.to_datetime(df_op[col]).dt.strftime('%d/%m/%Y')
    
    real_columns = ['Strike Put', 'Preço Put', 'Strike Call', 'Preço Call', 
                    'Preço Ativo', 'Custo Operação', 'Lucro Mínimo', 'Lucro Máximo']
    for col in real_columns:
        df_op[col] = df_op[col].apply(lambda x: "R$ " + str(x).replace('.', ',') if pd.notnull(x) else x)
    
    pct_columns = ['Risco (%)', 'Lucro Mín (%)', 'Lucro Máx (%)']
    for col in pct_columns:
        df_op[col] = df_op[col].apply(lambda x: str(x).replace('.', ',') + "%" if pd.notnull(x) else x)

    df_op = df_op.reindex(columns=['Ativo', 'Preço Ativo', 'Custo Operação', 'Vencimento',
                                   'Risco (%)', 'Lucro Mínimo', 'Lucro Mín (%)', 'Lucro Máximo', 'Lucro Máx (%)',
                                   'Ticker Put', 'Strike Put', 'Preço Put', 'Negócios Put', 'Último Negócio Put',
                                   'Ticker Call', 'Strike Call', 'Preço Call', 'Negócios Call', 'Último Negócio Call'])
    
    #df_op.reset_index(drop=True, inplace=True)
    
    
    return df, df_put, df_call, df_op


# Coleta as opções com todos os vencimentos
def opt_all(ativo):
    url = f'https://opcoes.net.br/listaopcoes/completa?idLista=ML&idAcao={ativo}&listarVencimentos=true&cotacoes=true'
    r = requests.get(url).json()
    vencimentos = [i['value'] for i in r['data']['vencimentos']]
    df = pd.concat([coleta_opcoes(ativo, vencimento) for vencimento in vencimentos])
    return df


# Lista as datas de vencimentos semanais
def vencimentos_opcoes():
    # Obter o mês e ano atuais
    mes_atual = datetime.now().month
    ano_atual = datetime.now().year
    ano_fim = ano_atual + 1
     
    sextas_feiras = []
    # Criar um objeto de feriados para o Brasil
    feriados = wd.get_holidays(country='BR', years=range(ano_atual, ano_fim))
    
    for ano in range(ano_atual, ano_fim):
        for mes in range(mes_atual, mes_atual + 13):
            # Ajustar o mês se ele ultrapassar 12
            if mes > 12:
                mes -= 12
                ano = ano_fim
            # Obter o primeiro dia do mês
            primeiro_dia = datetime(ano, mes, 1).date()
            # Encontrar a sexta-feira do mês
            sexta_feira = primeiro_dia + timedelta(days=(4 - primeiro_dia.weekday()) % 7)
            # Verificar se a sexta-feira é um feriado brasileiro
            while sexta_feira.month == mes:
                # Verificar se a sexta-feira é um feriado brasileiro
                if sexta_feira in feriados:
                    # Se for um feriado, retornar o dia útil anterior
                    sexta_feira = sexta_feira - timedelta(days=1)
                    while sexta_feira.weekday() >= 5: # Enquanto for fim de semana
                        sexta_feira -= timedelta(days=1)
                sextas_feiras.append(sexta_feira)
                # Avançar para a próxima sexta-feira
                sexta_feira += timedelta(days=7)
    hoje = datetime.now().date()
    vencimentos = [data for data in sextas_feiras if data > hoje]
    vencimentos = [data.strftime('%d/%m/%Y') for data in vencimentos]
    
    return vencimentos




In [71]:
ativos = ['ABCB4','ABEV3','AERI3','AESB3','AGRO3','ALOS3','ALPA4','ALUP11','AMAR3','AMBP3','AMER3','ANIM3','ARML3',
'ARZZ3','ASAI3','AURE3','AZUL4','B3SA3','BBAS3','BBDC3','BBDC4','BBSE3','BEEF3','BHIA3','BLAU3','BMGB4',
'BMOB3','BOVA11','BPAC11','BPAN4','BRAP4','BRFS3','BRKM5','BRSR6','CAML3','CASH3','CBAV3','CCRO3','CEAB3',
'CIEL3','CLSA3','CMIG4','CMIN3','COGN3','CPFE3','CPLE6','CRFB3','CSAN3','CSMG3','CSNA3','CURY3','CVCB3',
'CXSE3','CYRE3','DASA3','DIRR3','DXCO3','ECOR3','EGIE3','ELET3','ELET6','EMBR3','ENAT3','ENEV3','ENGI11',
'ENJU3','EQTL3','EVEN3','EZTC3','FESA4','FLRY3','GFSA3','GGBR4','GGPS3','GMAT3','GOAU4','GOLL4','GRND3',
'GUAR3','HAPV3','HBOR3','HBSA3','HYPE3','IBOV11','IFCM3','IGTI11','INTB3','IRBR3','ITSA4','ITUB3','ITUB4',
'IVVB11','JALL3','JBSS3','JHSF3','KEPL3','KLBN11','LEVE3','LIGT3','LJQQ3','LOGG3','LREN3','LWSA3','MBLY3',
'MDIA3','MEAL3','MGLU3','MILS3','MLAS3','MOVI3','MRFG3','MRVE3','MULT3','MYPK3','NEOE3','NTCO3','ODPV3',
'ONCO3','PCAR3','PETR3','PETR4','PETZ3','PNVL3','POMO4','POSI3','PRIO3','PSSA3','QUAL3','RADL3','RAIL3',
'RAIZ4','RANI3','RAPT4','RDOR3','RECV3','RENT3','ROMI3','RRRP3','SANB11','SAPR11','SBFG3','SBSP3','SEER3',
'SEQL3','SIMH3','SLCE3','SMAL11','SMFT3','SMTO3','SOMA3','SRNA3','STBP3','SUZB3','TAEE11','TASA4','TEND3',
'TIMS3','TOTS3','TRAD3','TRIS3','TRPL4','TTEN3','TUPY3','UGPA3','UNIP6','USIM5','VALE3','VAMO3','VBBR3',
'VIVA3','VIVT3','VLID3','VULC3','WEGE3','WIZC3','YDUQ3','ZAMP3']

In [72]:
ativo = 'VALE3'
vencimento = '2024-04-19'       # YYYY-MM-DD
quantidade = 1                 # quantidade é igual para o ativo, put e call
cdi, dias_uteis = calcula_cdi(vencimento)   # calcula o cdi da operação até o vencimento
volume_put = 0.01               # volume mínimo negociado em put
negocios_put = 1                # quantidade mínima de negócios em put
volume_call = 0.01              # volume mínimo negociado em call
negocios_call = 1               # quantidade mínima de negócios em call
filtro_data = '2024-02-29'      # retorna somente as opções negociadas após esse dia
risco = cdi         # risco máximo de perda. Pode ser qualquer valor ou vincular à variável cdi
corretagem_variavel = 0.00      # representa 0,50% de corretagem sobre o volume negociado
corretagem_ordem = 0.00         # no caso, corretagem gratuita
operacao = 'collar_alta'

mt5.initialize()
#ativos = ['PETR4', 'VALE3', 'ITUB4', 'MGLU3']
melhores_operacoes = []

# Itera sobre a lista de ativos
for ativo in ativos:
    # Chama a função para cada ativo e armazena o resultado
    try:
        df, df_put_alta, df_call_alta, df_op = collar(ativo, vencimento, operacao='collar_alta', 
                                                          filtro_data=filtro_data, risco=risco)
        # Verifica se df_op está vazio
        if not df_op.empty:
            # Se não estiver vazio, pega a melhor operação e a adiciona à lista
            melhor_operacao = df_op.iloc[0].to_frame().T
            melhores_operacoes.append(melhor_operacao)
            print(f'Operação encontrada em {ativo}, adicionando à lista.')
        else:
            
            # Se estiver vazio, simplesmente continua para o próximo ativo
            print(f"Nenhuma operação encontrada para {ativo}. Ignorando...")
    except Exception as e:
        # Se ocorrer uma exceção, imprime uma mensagem e pula para o próximo ativo
        print(f"Erro ao processar {ativo}: {e}. Ignorando e passando para o próximo ativo.")
    
    
# Concatena todos os DataFrames em uma única estrutura
try:
    df_operacoes_dia = pd.concat(melhores_operacoes, axis=0)
    df_operacoes_dia = df_operacoes_dia.sort_values(by='Lucro Mín (%)',ascending=False)
    df_operacoes_dia
except Exception as e:
    print(f"Nenhuma operação encontrada: {e}.")

mt5.shutdown()
'''df, df_put_alta, df_call_alta, df_op = collar(ativo, vencimento, operacao=operacao, 
                                                          filtro_data=filtro_data, risco=risco)

df_op'''

Erro ao processar ABCB4: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar ABEV3: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar AERI3: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar AESB3: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar AGRO3: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar ALOS3: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar ALPA4: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar ALUP11: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar AMAR3: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar AMBP3: 'Último Negócio Put'. Ignorando e passando para o próximo ativo.
Erro ao processar AMER3: 'Preço Put'. Ignorando e passando para o próximo ativo.
Erro ao processar 

'df, df_put_alta, df_call_alta, df_op = collar(ativo, vencimento, operacao=operacao, \n                                                          filtro_data=filtro_data, risco=risco)\n\ndf_op'

In [73]:
df_operacoes_dia

,Ativo,Preço Ativo,Custo Operação,Vencimento,Risco (%),Lucro Mínimo,Lucro Mín (%),Lucro Máximo,Lucro Máx (%),Ticker Put,Strike Put,Preço Put,Negócios Put,Último Negócio Put,Ticker Call,Strike Call,Preço Call,Negócios Call,Último Negócio Call
175,MGLU3,"R$ 2,17","R$ 2,17",19/04/2024,"1,41%","R$ 13,32","613,82%","R$ 15,82","729,03%",MGLUP15,"R$ 15,49","R$ 0,0",1.0,04/03/2024,MGLUD18,"R$ 17,99","R$ 0,0",1.0,04/03/2024
232,VALE3,"R$ 66,64","R$ 66,64",19/04/2024,"1,41%","R$ 34,09","51,16%","R$ 61,09","91,67%",VALEP110,"R$ 100,73","R$ 0,0",1.0,04/03/2024,VALED133,"R$ 127,73","R$ 0,0",3.0,01/03/2024
19739,PETR4,"R$ 40,31","R$ 40,31",19/04/2024,"1,41%","R$ 10,95","27,16%","R$ 20,45","50,73%",PETRP615,"R$ 51,26","R$ 0,0",1.0,01/03/2024,PETRD710,"R$ 60,76","R$ 0,0",21.0,04/03/2024
4157,ITUB4,"R$ 33,91","R$ 33,91",19/04/2024,"1,41%","R$ 6,94","20,47%","R$ 15,44","45,53%",ITUBP420,"R$ 40,85","R$ 0,0",1.0,01/03/2024,ITUBD505,"R$ 49,35","R$ 0,0",1.0,04/03/2024


In [66]:
df_op

,Ativo,Preço Ativo,Custo Operação,Vencimento,Risco (%),Lucro Mínimo,Lucro Mín (%),Lucro Máximo,Lucro Máx (%),Ticker Put,Strike Put,Preço Put,Negócios Put,Último Negócio Put,Ticker Call,Strike Call,Preço Call,Negócios Call,Último Negócio Call
175,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 13,27","597,75%","R$ 15,77","710,36%",MGLUP15,"R$ 15,49","R$ 0,0",1.0,01/03/2024,MGLUD18,"R$ 17,99","R$ 0,0",1.0,04/03/2024
89,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 10,77","485,14%","R$ 15,77","710,36%",MGLUP13,"R$ 12,99","R$ 0,0",1.0,01/03/2024,MGLUD18,"R$ 17,99","R$ 0,0",1.0,04/03/2024
863,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 9,02","406,31%","R$ 15,77","710,36%",MGLUP112,"R$ 11,24","R$ 0,0",1.0,01/03/2024,MGLUD18,"R$ 17,99","R$ 0,0",1.0,04/03/2024
3,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 8,77","395,05%","R$ 15,77","710,36%",MGLUP11,"R$ 10,99","R$ 0,0",1.0,01/03/2024,MGLUD18,"R$ 17,99","R$ 0,0",1.0,04/03/2024
5851,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 6,37","286,94%","R$ 15,77","710,36%",MGLUP464,"R$ 8,59","R$ 0,0",3.0,01/03/2024,MGLUD18,"R$ 17,99","R$ 0,0",1.0,04/03/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3083,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 0,07","3,15%","R$ 3,07","138,29%",MGLUP230,"R$ 2,29","R$ 0,0",26.0,04/03/2024,MGLUD530,"R$ 5,29","R$ 0,0",1.0,04/03/2024
3070,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 0,07","3,15%","R$ 1,87","84,23%",MGLUP230,"R$ 2,29","R$ 0,0",26.0,04/03/2024,MGLUD410,"R$ 4,09","R$ 0,0",1.0,04/03/2024
3069,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 0,07","3,15%","R$ 1,77","79,73%",MGLUP230,"R$ 2,29","R$ 0,0",26.0,04/03/2024,MGLUD399,"R$ 3,99","R$ 0,0",1.0,04/03/2024
3013,MGLU3,"R$ 2,22","R$ 2,22",19/04/2024,"1,41%","R$ 0,07","3,15%","R$ 15,77","710,36%",MGLUP230,"R$ 2,29","R$ 0,0",26.0,04/03/2024,MGLUD18,"R$ 17,99","R$ 0,0",1.0,04/03/2024


In [43]:
df # retorna o dataframe completo com todas as operações possíveis, independente de lucro ou prejuízo. Calcula para cada put, todas as calls

,Ativo,Vencimento,Ticker Put,Tipo Put,Strike Put,Negócios Put,Volume Put,Preço Put,Último Negócio Put,Ticker Call,...,Último Negócio Call,Preço Ativo,Custo Operação,Corretagem,Corretagem (%),Risco (%),Lucro Mínimo,Lucro Máximo,Lucro Mín (%),Lucro Máx (%)
0,MGLU3,2024-04-19,MGLUP11,PUT,10.99,1.0,104400.0,0.0,NaN,MGLUD11,...,NaN,2.22,2.22,0.0,0.0,1.41,8.77,8.77,395.05,395.05
1,MGLU3,2024-04-19,MGLUP11,PUT,10.99,1.0,104400.0,0.0,NaN,MGLUD13,...,NaN,2.22,2.22,0.0,0.0,1.41,8.77,10.77,395.05,485.14
2,MGLU3,2024-04-19,MGLUP11,PUT,10.99,1.0,104400.0,0.0,NaN,MGLUD15,...,NaN,2.22,2.22,0.0,0.0,1.41,8.77,13.27,395.05,597.75
3,MGLU3,2024-04-19,MGLUP11,PUT,10.99,1.0,104400.0,0.0,NaN,MGLUD18,...,NaN,2.22,2.22,0.0,0.0,1.41,8.77,15.77,395.05,710.36
4,MGLU3,2024-04-19,MGLUP11,PUT,10.99,1.0,104400.0,0.0,NaN,MGLUD21,...,NaN,2.22,2.22,0.0,0.0,1.41,8.77,18.77,395.05,845.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7391,MGLU3,2024-04-19,MGLUP900,PUT,0.89,NaN,NaN,0.0,NaN,MGLUD740,...,NaN,2.22,2.22,0.0,0.0,1.41,-1.33,5.17,-59.91,232.88
7392,MGLU3,2024-04-19,MGLUP900,PUT,0.89,NaN,NaN,0.0,NaN,MGLUD759,...,NaN,2.22,2.22,0.0,0.0,1.41,-1.33,5.37,-59.91,241.89
7393,MGLU3,2024-04-19,MGLUP900,PUT,0.89,NaN,NaN,0.0,NaN,MGLUD800,...,NaN,2.22,2.22,0.0,0.0,1.41,-1.33,-1.43,-59.91,-64.41
7394,MGLU3,2024-04-19,MGLUP900,PUT,0.89,NaN,NaN,0.0,NaN,MGLUD840,...,NaN,2.22,2.22,0.0,0.0,1.41,-1.33,5.87,-59.91,264.41


In [10]:
df_put_alta # retorna todas as puts disponíveis para o ativo

,Ativo,Vencimento,Ticker,Tipo,Strike,Negócios,Volume
31,BRKM5,2024-04-19,BRKMP120,PUT,12.00,1.0,4.0
32,BRKM5,2024-04-19,BRKMP130,PUT,13.00,1.0,20.0
33,BRKM5,2024-04-19,BRKMP140,PUT,14.00,1.0,15.0
34,BRKM5,2024-04-19,BRKMP150,PUT,15.00,2.0,76.0
35,BRKM5,2024-04-19,BRKMP155,PUT,15.50,2.0,153.0
36,BRKM5,2024-04-19,BRKMP160,PUT,16.00,3.0,90.0
37,BRKM5,2024-04-19,BRKMP165,PUT,16.50,6.0,201.0
38,BRKM5,2024-04-19,BRKMP170,PUT,17.00,15.0,4097.0
39,BRKM5,2024-04-19,BRKMP175,PUT,17.50,2.0,61.0
40,BRKM5,2024-04-19,BRKMP180,PUT,18.00,3.0,613.0


In [11]:
df_call_alta # retorna todas as puts disponíveis para o ativo

,Ativo,Vencimento,Ticker,Tipo,Strike,Negócios,Volume
0,BRKM5,2024-04-19,BRKMD120,CALL,12.00,1.0,9650.0
1,BRKM5,2024-04-19,BRKMD130,CALL,13.00,NaN,NaN
2,BRKM5,2024-04-19,BRKMD140,CALL,14.00,NaN,NaN
3,BRKM5,2024-04-19,BRKMD150,CALL,15.00,NaN,NaN
4,BRKM5,2024-04-19,BRKMD155,CALL,15.50,NaN,NaN
5,BRKM5,2024-04-19,BRKMD160,CALL,16.00,NaN,NaN
6,BRKM5,2024-04-19,BRKMD165,CALL,16.50,NaN,NaN
7,BRKM5,2024-04-19,BRKMD170,CALL,17.00,2.0,4550.0
8,BRKM5,2024-04-19,BRKMD175,CALL,17.50,1.0,280.0
9,BRKM5,2024-04-19,BRKMD180,CALL,18.00,10.0,7533.0


In [81]:
#OPÇÕES
bot = telebot.TeleBot("6832366281:AAF8pcibmxZGWyPUeGHgHOT64qkMgiKGCQA")
grupo = "-1001803272086"
data_hoje = date.today().strftime('%d/%m/%Y')

for index, row in df_operacoes_dia.iterrows():
    message = (
        f"🟢COLLAR DE ALTA {row['Ativo']}\n"
        f"🕔Validade: {data_hoje} 16:45\n"
        f"💰Aporte: {row['Custo Operação']}\n"
        f"🚀Lucro Max: {row['Lucro Máximo']} ({row['Lucro Máx (%)']})\n"
        f"❗Risco: {row['Lucro Mínimo']} ({row['Lucro Mín (%)']})\n"
        "-----------------------------------\n"
        "\n"
        "💱OPERAÇÃO\n"
        f"➕{row['Ativo']}: {row['Preço Ativo']}\n"
        f"➕PUT {row['Ticker Put']}: Strike {row['Strike Put']} ({row['Preço Put']})\n"
        f"➖CALL {row['Ticker Call']}: Strike {row['Strike Call']} ({row['Preço Call']})\n"
        "🟰Compra Ativo, Compra Put e Vende Call\n"
        f"🗓️Vencimento: {row['Vencimento']} ({dias_uteis}d.u.)\n"
        
    )

        # Enviando a mensagem para o canal
    bot.send_message(grupo, message, protect_content=True, message_thread_id=75)
    sleep(12)